# Recurrent Neural Network w/ `tensorflow`

In [4]:
import os.path
import random
import datetime

import numpy as np
import tensorflow as tf

## data file path

In [7]:
data_dir = '../datasets/wikitext-2-raw'
train_file = 'wiki.train.raw'
test_file = 'wiki.test.raw'

## read training data

In [13]:
data = open(os.path.join(data_dir, train_file), 'r').read()
print('Number of characters is {:,}'.format(len(data)))
print(data[:1000])

Number of characters is 10,918,892
 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making 

## pre-processing

In [15]:
chars = sorted(list(set(data)))
char_size = len(chars)
print('Char size: {:,}'.format(char_size))
print(chars)

Char size: 1,013
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}', '~', '\x93', '\x94', '¡', '¢', '£', '¥', '§', '°', '±', '²', '³', '´', 'µ', '·', 'º', '½', '¿', 'Á', 'Ä', 'Å', 'Æ', 'Ç', 'É', 'Í', 'Î', 'Ñ', 'Ó', 'Ö', '×', 'Ø', 'Ú', 'Ü', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'Ā', 'ā', 'ă', 'ą', 'ć', 'Č', 'č', 'Đ', 'đ', 'Ē', 'ē', 'Ĕ', 'ĕ', 'ė', 'ę', 'ğ', 'ħ', 'ī', 'İ', 'ı', 'Ľ', 'Ł', 'ł', 'ń', 'Ō', 'ō', 'œ', 'ř', 'Ś', 'ś', 'Ş', 'ş', 'Š', 'š', 'ţ', 'ũ', 'ū',

In [16]:
char_2_idx = {ch: i for i,ch in enumerate(chars)}
idx_2_char = {i: ch for i,ch in enumerate(chars)}

## helper methods
### Generate probability of each next char

In [ ]:
def sample(prediction):
    r = np.random.uniform(0, 1)
    s = 0